In [1]:
# importazione delle librerie
import pandas as pd
import pandas_ta as ta
import numpy as np
import funzioni as fn

# importiamo il path
from funzioni import PATH, PATH_ESTR

CREIAMO LA NOSTRA CLASSE PER IL GIOCO DEL LOTTO

In [2]:
# funzione per dire quando e' uscito un numero
def uscite(df, numero):
    # prendiamo l'indice della riga che contiene un valore in una determinata colonna
    uscite = np.array([])
    for i in df.columns[2:]:
        # aggiungi l'indice della riga che contiene il valore 58 nella colonna i
        uscite = np.append(uscite, df[df[i] == str(numero)].index.values.astype(int))
    # aggiungiamo ad ogni valore della lista il valore 1
    uscite = np.sort(uscite + 1)

    return uscite


In [3]:
def ritardi(df, numero):

    u = uscite(df, numero)
    # calcuiamo la differenza tra ogni valore della lista e il successivo
    ritardi = np.diff(u)
    # inseriamo al primo posto il valore 0
    ritardi = np.insert(ritardi, 0, 0)

    return ritardi

In [76]:
class Lotto:

    def __init__(self, path=PATH_ESTR):
        """Di default il path e' la cartella estrazioni"""
        self.ruota = None
        self.df = None
        self.numero = None
        self.path = path

    def carica_estrazioni(self, ruota):
        r = self.path+ruota+'.csv'
        estrazioni = fn.leggi_da_csv(r)
        df = pd.DataFrame(estrazioni[1:], columns=estrazioni[0])
        self.ruota = ruota
        return df

    def carica_numero(self, numero, df):

        self.df = df
        self.numero = numero
        
        # calcoliamo le posizioni di indice delle uscite del numero
        self.elenco_uscite = uscite(self.df, self.numero)
        # il numero totali delle volte che e' uscito e' dato dalla lunghezza della lista
        self.n_uscite = len(self.elenco_uscite)
        # il totale delle estrazioni e' dato dalla lunghezza del dataframe
        self.totale_estrazioni = len(self.df)
        # il vettore delle frequenze e' dato dal vettore del numero di uscite diviso il vettore dell'indice del vettore stesso + 1
        self.frequenze = self.elenco_uscite / (np.arange(len(self.elenco_uscite))+1)
        # la frequenza media (ultima frequenza o frequenza attuale) e' data dal totale delle estrazioni diviso il numero di uscite
        self.ultima_frequenza = round(float(self.totale_estrazioni/self.n_uscite), 2)
        # l'elenco dei ritardi e' dato dal vettore dei ritardi
        self.ritardi = ritardi(self.df, self.numero)
        # il ritardo attuale e' dato dalla differenza tra il totale delle estrazioni e l'indice l'ultima uscita
        self.ritardo_attuale = self.totale_estrazioni - self.elenco_uscite[-1]
        # calcoliamo il ritardo massimo storico
        self.ritardo_massimo = self.ritardi.max()
        # salviamo l'ultimo ritardo
        self.ultimo_ritardo = self.ritardi[-1]
        # calcoliamo la scompensazione
        self.scompensazioni = self.ritardi - self.frequenze
        # calcoliamo la scompensazione attuale e l'ultima scompensazione
        self.ultima_scompensazione = self.scompensazioni.sum()
        self.scompensazione_attuale = self.ultima_scompensazione + (self.ritardo_attuale - self.ultima_frequenza)
        # salviamo le statistiche in un dizionario
        self.statistiche = {
            'ruota': self.ruota,
            'numero': self.numero,
            'n_uscite': self.n_uscite,
            'totale_estrazioni': self.totale_estrazioni,
            'ultima_frequenza': self.ultima_frequenza,
            'ritardo_attuale': self.ritardo_attuale,
            'ritardo_massimo': self.ritardo_massimo,
            'ultimo_ritardo': self.ultimo_ritardo,
            'ultima_scompensazione': self.ultima_scompensazione,
            'scompensazione_attuale': self.scompensazione_attuale
        }
    def stampa_statistihe(self):
        
        for k, v in self.statistiche.items():
            print(f'{k}: {v}')

    def calcolo_trend(self):
        # utilizzo le funzioni di pandas_ta per calcolare il trend della scompensazione
        df = pd.DataFrame()
        df['scompensi'] = self.scompensazioni
        # aggiungo la colonna close perche' la funzione ta.sma la richiede
        df['close'] = df['scompensi'].cumsum()
        sma10 = df.ta.sma(length=10)
        sma30 = df.ta.sma(length=30)
        df = pd.concat([df, sma10, sma30], axis=1)
        df['scomp_sigLine'] = np.where(df['SMA_10'] > df['SMA_30'], 1, 0 )
        df['scomp_position'] = df['scomp_sigLine'].diff()
        df["Status"] = np.where(df['scomp_sigLine'] == 0, 'F', 'R')
        self.trend = df['Status'].iloc[-1]
        self.statistiche['trend'] = self.trend
    
    def previsioni(self, ruota):

        temp = self.carica_estrazioni(ruota)
        df = pd.DataFrame()

        # calcoliamo le previsioni
        for i in range(1, 91):

            self.carica_numero(i, temp)
            self.calcolo_trend()
            df = df.append(self.statistiche, ignore_index=True)
        
        df = df[(df['ritardo_attuale'] < 15) & (df['scompensazione_attuale'] < 0) & (df['trend'] == 'R')].sort_values(by='ultima_frequenza', ascending=False)
        self.previsione = df['numero'].values
        
        return f"i numeri da giocare su {ruota} sono: {self.previsione}"
        
            


In [79]:
# leggiamo il file csv delle estrazioni di Bari
a = Lotto()

ruote = ['BA', 'CA', 'FI', 'GE', 'MI', 'NA', 'PA', 'RM', 'TO', 'VE', 'RN']

for i in ruote:
    df = a.carica_estrazioni(i)
    print(a.previsioni(i))

i numeri da giocare su BA sono: [ 7. 42. 67. 82. 31. 77. 11. 83. 14. 70. 61. 87. 81. 34. 40. 48.]
i numeri da giocare su CA sono: [86. 58. 74.  6. 65. 69. 16. 77. 90.  1.]
i numeri da giocare su FI sono: [49. 15. 43. 42. 17. 81. 83.  8. 28. 90.  2. 13. 10. 69. 57. 74. 79.]
i numeri da giocare su GE sono: [64. 23. 49. 12.  1. 89. 35. 53. 86. 44. 71. 47. 84.]
i numeri da giocare su MI sono: [63. 34. 70. 73. 74. 28. 19. 56. 66. 31.  8. 88. 55. 35. 79.]
i numeri da giocare su NA sono: [62. 79. 60. 65.  8. 82. 23. 69. 80. 45.]
i numeri da giocare su PA sono: [ 1. 79. 11. 80. 33. 66. 28. 17. 74. 45. 90.]
i numeri da giocare su RM sono: [23. 26. 28.  1. 29. 17. 19. 20. 40. 67. 88. 31. 11.  8. 43.]
i numeri da giocare su TO sono: [ 3. 52. 28.  6. 10. 90. 84.  9. 54. 85.]
i numeri da giocare su VE sono: [26. 69. 16. 67.  5.  3. 79. 87. 30. 43. 38. 66. 84. 74.]
i numeri da giocare su RN sono: [41. 19. 56. 73. 59. 43. 29. 82. 10. 25. 50. 89. 33. 80.]
